<a href="https://colab.research.google.com/github/Lucs1590/image_challenge/blob/master/src/exercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install GDAL
import operator
import cv2
from osgeo import gdal,ogr,osr
from google.colab import drive

In [2]:
drive.mount('/content/drive')
raster = gdal.Open(r'/content/drive/MyDrive/Outros/talhoes_tifs/4039_01_04 (1).tif')
projection = raster.GetProjection()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 1
srs = osr.SpatialReference(wkt=projection)
if srs.IsProjected:
    print(srs.GetAttrValue('projcs'))
print("Coordinate system: {0}".format(srs.GetAttrValue('geogcs')))

SIRGAS 2000 / UTM zone 22S
Coordinate system: SIRGAS 2000


In [4]:
# 3
print("Image size: {0} pixels x {1} pixels (X x Y)".format(raster.RasterXSize, raster.RasterYSize))

Image size: 2396 pixels x 3247 pixels (X x Y)


In [5]:
# 4
raster_geo_tr = raster.GetGeoTransform()
raster_proj_ref = raster.GetProjectionRef()
raster_spatial_ref = osr.SpatialReference(raster_proj_ref)
geo_srs = raster_spatial_ref.CloneGeogCS()
transform = osr.CoordinateTransformation(raster_spatial_ref, geo_srs)
raster_bbox_cells = (
    (0., 0.),
    (0, raster.RasterYSize),
    (raster.RasterXSize, raster.RasterYSize),
    (raster.RasterXSize, 0),
)

for x, y in raster_bbox_cells:
    x2 = raster_geo_tr[0] + raster_geo_tr[1] * x + raster_geo_tr[2] * y
    y2 = raster_geo_tr[3] + raster_geo_tr[4] * x + raster_geo_tr[5] * y
    geo_pt = transform.TransformPoint(x2, y2)[:2]
    print("X: {0}, Y: {1}".format(x, y), '->' , "Latitude: {0}, Longitude: {1}".format(geo_pt[0], geo_pt[1]))


X: 0.0, Y: 0.0 -> Latitude: -49.70197280463707, Longitude: -19.059188492405724
X: 0, Y: 3247 -> Latitude: -49.70194997381456, Longitude: -19.062122252673113
X: 2396, Y: 3247 -> Latitude: -49.69967322916704, Longitude: -19.06210621797093
X: 2396, Y: 0 -> Latitude: -49.69969610000672, Longitude: -19.059172460356496


In [6]:
# 2
print("Resolution: {0} cm/px".format(round(x2/raster.RasterXSize*0.1, 2)))

Resolution: 26.58 cm/px


In [7]:
# 5
def make_windows(img, img_size, path):
    x = 0
    for i in range(img_size, img.shape[0], img_size):
        for j in range(img_size, img.shape[1], img_size):
            cv2.imwrite(
                "{0}{1}.png".format(path, x), img[i-img_size:i, j-img_size:j])
            x += 1
    return img

In [8]:
img_1 = cv2.imread(r"/content/drive/MyDrive/Outros/talhoes_tifs/talhao_ponkan.tif")
img_2 = cv2.imread(r"/content/drive/MyDrive/Outros/talhoes_tifs/44359_01 (2).tif")
img_3 = cv2.imread(r"/content/drive/MyDrive/Outros/talhoes_tifs/talhao_santana2.tif")

make_windows(img_1, 256, "/content/drive/MyDrive/Outros/talhoes_tifs/results2/")
make_windows(img_2, 256, "/content/drive/MyDrive/Outros/talhoes_tifs/results3/")
make_windows(img_3, 256, "/content/drive/MyDrive/Outros/talhoes_tifs/results4/")

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)